<a href="https://colab.research.google.com/github/jmartin1976/BFA_samplescripts/blob/main/CF_InfoRetrievalfromCohorts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Functions to retrieve content and structure it from company candidates to Catalist Fund

### Next steps
* Store JSON information in CSVs
* Retrieve for each founder
  gender
  first time founder
  university

In [271]:
!pip install langchain
!sudo apt-get update
!sudo apt-get -qq install poppler-utils tesseract-ocr
# Upgrade Pillow to latest version
%pip install -q --user --upgrade pillow
# Install Python Packages
%pip install -q unstructured["local-inference"]==0.8.4
!pip install kora

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [849 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [834 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 2,020 kB in 2s (1,114 kB/s)
Reading package lists... D

In [276]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [277]:
!pip install openai
import pandas as pd
import os

import requests
from requests.exceptions import InvalidURL

import io
from io import BytesIO

from bs4 import BeautifulSoup
from urllib.parse import urlparse

from google.colab import auth
from google.colab import drive
import gspread
from google.auth import default

#import re
import openai

#from transformers import GPT2Tokenizer
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_loaders import UnstructuredURLLoader

from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaIoBaseUpload

from kora.xattr import get_id

In [278]:
# Add your Bing Search V7 subscription key to your environment variables.
# SUBSCRIPTION_KEY = os.environ['BING_SEARCH_V7_SUBSCRIPTION_KEY']
# ENDPOINT = os.environ['BING_SEARCH_V7_ENDPOINT']
bing_subscription_key = "cb9f53c0a43e42c99eb31985e0e67baf"
bing_search_url = "https://api.bing.microsoft.com"+"/v7.0/search"
#search_term = "Nneile Nkholise where did she studied?"
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
def is_domain_fromName(name,domain):
  name_parts = name.split()
  name_slug = ''.join(name_parts).lower()
  print(name_slug)
  # Compare the two strings
  if name_slug == domain:
    return 1
  else:
    return 0

def is_domain_fromEmail(email,domain):
  if ('@' not in email) or (not email):
  # Handle the case where the email string does not contain a '@' character
    return 0
  email_domain = email.split('@')[1].split('.')[0]
  # Compare the two strings
  if email_domain == domain:
    return 1
  else:
    return 0

In [ ]:
def get_URL_fromNameEmail(name,email,context,bing_subscription_key,bing_search_url):
  headers = {"Ocp-Apim-Subscription-Key": bing_subscription_key}
  query=name+context
  params = {"q": query,
          "textDecorations": True,
          "textFormat": "HTML",
          "count": 3}
  response = requests.get(bing_search_url, headers=headers, params=params)
  response.raise_for_status()
  search_results = response.json()

  # Print the search results
  for result in search_results["webPages"]["value"]:
    parsed_url = urlparse(result["url"])
    domain_parts = parsed_url.netloc.split('.')
    second_level_domain = domain_parts[-2]
    # print(result["url"])
    if is_domain_fromName(name,second_level_domain):
      # print(f"{second_level_domain} matches {name}")
      return result["url"]
    else:
      if is_domain_fromEmail(email,second_level_domain):
        # print(f"{second_level_domain} matches {email}")
        return result["url"]
  return("no url found")

In [ ]:
get_URL_fromNameEmail("TechFusion Africa","matt@techfsn.com","",bing_subscription_key,bing_search_url)

techfusionafrica


'https://www.techfsn.com/'

In [ ]:
# opens filepath, look for companyname row and retrieve candidate urls from bing based on Company Name and basic string comparison
# stores the url_candidate in a new row URL_candidate
def get_URL_candidates(filepath,sheetname,content):
  auth.authenticate_user()
  creds, _ = default()
  # uses credentials of the user running the script
  gc = gspread.authorize(creds)
  sh = gc.open_by_url(filepath)
  worksheet = sh.worksheet(sheetname)
  # get_all_values gives a list of rows.
  rows = worksheet.get_all_values()
  df=pd.DataFrame.from_records(rows,columns=worksheet.row_values(1))
  # get column names
  column_names = df.columns.tolist()
  # drop column names from the dataframe
  df = df.drop(df.index[0])
  urls = []
  #for _,row in df.iloc[158:163].iterrows():
  for _,row in df.iterrows():
    # Obtain URL from document using bing
    companyname=row["Startup Name"]
    email=row["Email"]
    url=""
    url=get_URL_fromNameEmail(companyname,email,"",bing_subscription_key,bing_search_url)
    # print and add URL to the list
    # print(f"For {companyname} the url bing provided is : {url}")
    # Append the URL to the Google Sheet
    urls.append(url)
  #creates a column candidate url in the dataframe
  df['candidate url'] = urls
  # updates google sheet with new values in the dataframe
  values = df.values.tolist()
  # crean content in the google sheet
  worksheet.clear()
  # updates google sheets with new values
  worksheet.update(values)
  column_names = df.columns.tolist()
  #inset back column names
  worksheet.insert_row(column_names, 1)
  return rows

In [268]:
def get_URL_content(url):
  # create an id
  # store the content in id_webcontent.txt
  # get the url
  # save id in id column
  # save a link to id_webcontent in column url_content
  try:
    if "://" not in url or url.count('/') < 3:
      print(f"Invalid URL: {url}")
      return None
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup.get_text()
  except InvalidURL as e:
    print(f"An error occurred while processing the URL {url}: {e}")
    return None
  except Exception as e:
    print(f"An error occurred while processing the URL {url}: {e}")
    return None

In [ ]:
print(get_URL_content("https://cleantechnologyhub.com/"))

In [ ]:
filepath='https://docs.google.com/spreadsheets/d/1n4cB9QQwWvAVXlz9XEXFJ1Skv7Jabl32I1qHvennZus/edit#gid=567602188'
sheetname="K12 Pipeline"

In [232]:
def PDF2txt(folderpath):
  auth.authenticate_user()
  creds, _ = default()
  service = build('drive', 'v3', credentials=creds)
  #read from a folder all the pdfs, names should include an identifier to relate to the company
  pdf_files = [f for f in os.listdir(pdf_folder_path) if (f.endswith('.pdf')or(f.endswith('.pptx'))]
  for pdf_file in pdf_files:
    print(pdf_file)
    file_path = os.path.join(folderpath, pdf_file)
    loader = UnstructuredFileLoader(file_path)
    docs = loader.load()
    content=docs[0].page_content[:]
    #get the folderid and creates the txt in the same folder, remove the extension pdf
    folder_id = get_id(folderpath)
    file_metadata = {'name': f"{pdf_file}.txt", "parents": [folder_id],'mimeType': 'text/plain'}
    media = MediaIoBaseUpload(BytesIO(content.encode()), mimetype='text/plain')
    # Create the file in Google Drive
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print(f'File ID: {file.get("id")}')

In [ ]:
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
pdf_folder_path = f'{root_dir}/CF_PDF/'

In [233]:
print(PDF2txt(pdf_folder_path))

Tolbi - Pitch Deck.pdf
File ID: 1oVKNQkUSahR0-fvvJaFMJNEtTM5BPhKQ
Bio-Logical_20230516.pdf
File ID: 1aC0S7zLy-_uYtx6wBlAQnDoqvXomkoTo
Thola_20230523.pdf
File ID: 1gB4F6YFpW297taFvFY4BskXHTyhi0pAG
NoorNation_202305_Pitch deck.pdf
File ID: 1me1OvPYAti83lp7ke5ZJo-OmOtuo9JZG
None


In [279]:
def ask_GPT_on_txtfile(folderpath,query,format):
  #how to remove this from here
  openai.api_key=os.environ["OPENAI_API_KEY"]
  auth.authenticate_user()
  creds, _ = default()
  service = build('drive', 'v3', credentials=creds)
  #read from a folder all the pdfs, names should include an identifier to relate to the company
  txt_files=""
  txt_files = [f for f in os.listdir(pdf_folder_path) if f.endswith('.txt')]
  for txt_file in txt_files:
    print(txt_file)
    file_path = os.path.join(folderpath, txt_file)
    loader = UnstructuredFileLoader(file_path)
    docs = loader.load()
    content=docs[0].page_content[:]
    try:
      # Code to make API request goes here
      response = openai.ChatCompletion.create(
        #model="gpt-3.5-turbo",
        #use 16k version in order to allow processing the docs
        model='gpt-3.5-turbo-16k',
        messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": f"{content}\n\n{query}\n\n{format}"},
        ]
      )
      print(response['choices'][0]['message']['content'])
    except Exception as e:  # Catch all exceptions
      error_message = str(e)
      if "maximum context length" in error_message:
        print("The request is too long for the model:", e)
      else:
        print("An error ocurred with openAI:", e)


In [280]:
query='''Please extract the following information:
user_type: what users are they serving
technology: which technology they use
problem: the problem that the startup is working on
stage: Stage of the company or current fundraise (pre-see, seed, seriesA)
founder name. Include up to 3 founder names
If the information is not available in the dec, write NA'''
format='''generate a JSON-formatted string in this format
{\"user_type\": \"value\",
\"technology\": \"value\",
\"problem\": \"value\",
\"stage\": \"value\",
\"founder1\": \"value\",
\"founder2\": \"value\",
\"founder3\": \"value\"}'''
print(ask_GPT_on_txtfile(pdf_folder_path,query,format))

Tolbi - Pitch Deck.pdf.txt
{"user_type": "Small scale producers, Agri businesses, Local communities",
"technology": "Cloud-based AI decision-making platform using satellite images",
"problem": "Lack of data leading to increased production costs, low income for farmers, water waste and low yield, fertilizer waste, post-harvest losses",
"stage": "Seed",
"founder1": "Mouhamadou L. KEBE",
"founder2": "Daouda SECK",
"founder3": "Cherif Younouss NDIAYE"}
Bio-Logical_20230516.pdf.txt
{"user_type": "Climate-vulnerable communities in Africa",
"technology": "Nature-based carbon removals and biochar production",
"problem": "Climate change impacts on African smallholders, including temperature rise, droughts, and soil degradation",
"stage": "Seed",
"founder1": "Philip Hunter",
"founder2": "Rory Buckworth",
"founder3": "NA"}
NoorNation_202305_Pitch deck.pdf.txt
{"user_type": "Small & Medium Farming & Touristic Businesses",
"technology": "Decentralized green infrastructure",
"problem": "Lack of acce